# Assignment: Analyzing Airline Flight Delays 
#### By Brett Hallum, Chris Ficklin, and Ryan Shuhart<br>April 2017

We worked with the airline data set with a goal to use Python and its libraries to manage this large data set with out-of-core memory.

We wanted answers the following questions with use of the split-apply-combine technique:
* Which airports are most likely to be delayed flying out of or into?
* Which flights with same origin and destination are most likely to be delayed?
* Can you regress how delayed a flight will be before it is delayed?
* What are the most important features for this regression?

Within the scope of answering these questions, the models were cross-validated using sampling techniques and evalutated using criteria and standards set up through the FAA.


In [1]:
import dask.dataframe as dd #http://dask.pydata.org/en/latest/
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.io import output_notebook

total_time = datetime.now()

# from dask.distributed import Client
# client = Client(set_as_default=True)
# print(client)

### Other Settings
# Show more rows
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# Prevent scientific notation of decimals
pd.set_option('precision',3)
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
# Allow inline display of bokeh graphics
output_notebook()

Loading BokehJS ...

## Dask

Dask is a partial implementation of Pandas to split a large dataframe into many smaller Pandas dataframes. This library uses parallel computation for its analytics and is composed of a task scheduler and Big Data collections, such as the dataframe we will use. Dask is fast, flexible and scalable for many clusters with 1000’s of cores. Dask is easy to use because it mimics a lot of the API syntax of pandas and numpy to calculate various values. The data frames and groupby functions are nearly identical between dask and pandas.

## Data

The flight data contained 29 variables with various data types. In order to use Dask in an appropriate manner, these data types all needed to be converted to numeric values. This is done further down below after reading in the data from the parquet files.

In [2]:
# http://stat-computing.org/dataexpo/2009/the-data.html
var_desc = pd.read_csv("../ref/var_descriptions.csv", index_col='var_id')
var_desc

,Name,Data Type,Description
var_id,,,
1,Year,int64,1987-2008
2,Month,int64,1 - 12
3,DayofMonth,int64,1 - 31
4,DayOfWeek,int64,1 (Monday) - 7 (Sunday)
5,DepTime,float64,"actual departure time (local, hhmm)"
6,CRSDepTime,int64,"scheduled departure time (local, hhmm)"
7,ArrTime,float64,"actual arrival time (local, hhmm)"
8,CRSArrTime,int64,"scheduled arrival time (local, hhmm)"
9,UniqueCarrier,O,unique carrier code


In [ ]:
# Data Location
#parq_folder = "../data/parquet-tiny/" # Testing
#parq_folder = "../data/parquet_25/" # Higher Load Testing
parq_folder = "../data/parquet/" # Full data

# Load compressed Parquet format of all years ~2 sec
start = datetime.now()
# All data
df = dd.read_parquet(parq_folder+"processed/")

# Data 1995 and after with available tail numbers of planes
df_with_tails = dd.read_parquet(parq_folder+"processed_age/")
print("Load parquet time: ", datetime.now() - start)
print()

# Length of dask dataframe ~3 min
start = datetime.now()
# print("There are {:,d} observations in all the data.".format(len(df))) #123,534,969 Matches Eric Larson
# print("There are {:,d} observations after 1994.".format(len(df_with_tails))) 
print("Time to determine row counts: ", datetime.now() - start)

The original data set was pulled in a CSV format. We wanted to convert to the parquet format in order to better store the data and analyze it within Dask. Parquet is a binary data store format that is columnar storage. It allows data to be split and run in parallel. This conversion can be seen in Appendix A.

We load all of the data into a single dask dataframe for use within the rest of the analysis.

### Glance at Beginning and End of Dask Dataframe

In [14]:
print("First 5 rows:")
df.head()

First 5 rows:


,Year,Month,DayOfWeek,DepTime,CRSDepTime,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Distance,Hour,FlightAge,Hour_scaled,Distance_scaled
0,1987,10,3,741.000,730,PS,None,23.000,11.000,SAN,SFO,447.000,7.000,23853,-1.281,-0.462
1,1987,10,4,729.000,730,PS,None,14.000,-1.000,SAN,SFO,447.000,7.000,23853,-1.281,-0.462
2,1987,10,6,741.000,730,PS,None,29.000,11.000,SAN,SFO,447.000,7.000,23853,-1.281,-0.462
3,1987,10,7,729.000,730,PS,None,-2.000,-1.000,SAN,SFO,447.000,7.000,23853,-1.281,-0.462
4,1987,10,1,749.000,730,PS,None,33.000,19.000,SAN,SFO,447.000,7.000,23853,-1.281,-0.462


In [15]:
print("Last 5 rows:")
df_with_tails.head()

Last 5 rows:


,Year,Month,DayOfWeek,DepTime,CRSDepTime,UniqueCarrier,TailNum,ArrDelay,DepDelay,Origin,Dest,Distance,Hour,FlightAge,FirstFlight,Age,Hour_scaled,Distance_scaled,Age_scaled
0,1995,1,5,657.000,645,UA,N7298U,15.000,12.000,ORD,PHL,678.000,6.000,23940,23940,0,-1.471,-0.089,nan
1,1995,1,6,648.000,645,UA,N7449U,1.000,3.000,ORD,PHL,678.000,6.000,23940,23940,0,-1.471,-0.089,nan
2,1995,1,7,649.000,645,UA,N7453U,-5.000,4.000,ORD,PHL,678.000,6.000,23940,23940,0,-1.471,-0.089,nan
3,1995,1,1,645.000,645,UA,N7288U,-9.000,0.000,ORD,PHL,678.000,6.000,23940,23940,0,-1.471,-0.089,nan
4,1995,1,2,645.000,645,UA,N7275U,-6.000,0.000,ORD,PHL,678.000,6.000,23940,23940,0,-1.471,-0.089,nan


## Feature Preparation and Creation

In preparing the files for analysis, we also wanted to create new features that could be used throughout the rest of the processing. There are 2 features created for the regression with 2 additional features as intermediaries. The first feature is the Hour variable. As visualized in a following section, later hours have more delays. It is transformed from the scheduled departure time (CRSDepTime) to the hour of the day. 

The other feature created is the estimated plane age. The age could be a potential factor in delays and will be used during the regression analysis of the data. The age of a plane is estimated from the day of the first flight found in the data. It is calculated by determining the number of months from 0 A.D. for each flight, and the first flight of each plane. The difference results in the number of months between the first flight and each flight, or the estimated plane age in months. 

In addition, Hour, Distance, and Age are standardized to a z-score as new fields in order to determine the comparable feature importance.  

For faster processing of the regression, a processed data set is created after the features and z-score scaling. The code for the features and scaling can be located in Appendix D.

## Flight Delays

According to the FAA, when a schedule airflight departs more than 15 minutes after its scheduled time, it is considered officially delayed. We utilize the same logic for arrival times to determine if a flight is arriving late and is therefore delayed by arrival rather than by departure. Only departures and arrivals 15 minutes past the scheduled time will be considered late in the analysis.

http://aspmhelp.faa.gov/index.php/Types_of_Delay

### Aggregations

When running Dask, you can view visualizations of the distrubuted work by going to the host and port below. This uses the bokehJS library which watches the computer's cores and gives visual feedback of what is happening on all of the cores while also tracking the list of completed and non-completed tasks for the given block of code.

Below we note the conversion of several of the variables to numeric values with the use of categorization. This helps make the data set fully numeric so Dask runs in the correct manner.

http://127.0.0.1:8787/

In [ ]:
import dask
start = datetime.now()

# Make Categories as categorical. Making categorical reduces run time from ~10min to ~3min 
# with the process of making the variables categorical
df = df.categorize(['DayOfWeek', 'UniqueCarrier', 'Dest', 'Origin'])

# Define some aggregations to plot
aggregations = (
    #1 Average departure delay by year
    df.groupby('Year').DepDelay.mean(),
    
    #2 Average departure delay by Month
    df.groupby('Month').DepDelay.mean(), 
    
    #3 Average departure delay by hour of day
    df.groupby('Hour').DepDelay.mean(), 
    
    #4 Average departure delay by Carrier, top 15
    df.groupby('UniqueCarrier').DepDelay.mean().nlargest(15), 
    
    #5 Average arrival delay by destination, top 15
    (df.groupby('Dest').ArrDelay.mean().nlargest(15) 
     .reset_index().rename(columns={'ArrDelay':'AvgArrDelay'})),
    
    #6 Count of arrivals to destinations, excludes missing
    (df.groupby('Dest').ArrDelay.count() 
     .reset_index().rename(columns={'ArrDelay':'ArrCount'})),
    
    #7 Average departure delay by origin, top 15
    (df.groupby('Origin').DepDelay.mean().nlargest(15).reset_index().rename(columns={'DepDelay':'AvgDepDelay'})),
    
    #8 Count of departures by origin, excludes missing
    (df.groupby('Origin').DepDelay.count().reset_index().rename(columns={'DepDelay':'DepCount'})), 
    
    #9 Average departure by origin and destination
    (df.groupby(['Origin','Dest']).DepDelay.mean().reset_index().rename(columns={'DepDelay':'AvgDepDelay'})),
    
    #10 Count of departures between origin and destination
    (df.groupby(['Origin','Dest']).DepDelay.count().reset_index().rename(columns={'DepDelay':'DepCount'})),
    
    #11 Percentage of officially delayed flights by origin
    ((df[df.DepDelay>15].groupby('Origin').DepDelay.count() / df.groupby('Origin').DepDelay.count())
     .reset_index().rename(columns={'DepDelay':'PercDepDelay'})),
    
    #12 Percentage of officially late flights by destination
    ((df[df.ArrDelay>15].groupby('Dest').ArrDelay.count() / df.groupby('Dest').ArrDelay.count())
     .reset_index().rename(columns={'ArrDelay':'PercArrDelay'})),
                
    #13 Percentage of officially delayed flights by origin and destination
    ((df[df.DepDelay>15].groupby(['Origin','Dest']).DepDelay.count() / df.groupby(['Origin','Dest']).DepDelay.count())
     .reset_index().rename(columns={'DepDelay':'PercDepDelay'})),
                
    #14 Percentage of officially late flights by origin and destination
    ((df[df.ArrDelay>15].groupby(['Origin','Dest']).ArrDelay.count() / df.groupby(['Origin','Dest']).ArrDelay.count())
     .reset_index().rename(columns={'ArrDelay':'PercArrDelay'})),
    
    #15 Average departure delay by hour of day
    df.groupby('DayOfWeek').DepDelay.mean()
)

# Compute them all in a single command
(
delayed_by_year, #1
delayed_by_month, #2
delayed_by_hour, #3
delayed_by_carrier, #4
delayed_by_dest, #5
delayed_by_dest_count, #6
delayed_by_origin, #7
delayed_by_origin_count, #8
delayed_by_origin_dest, #9
delayed_by_origin_dest_count, #10
pct_delayed_by_origin, #11
pct_late_by_dest, #12
pct_delayed_by_origin_dest, #13
pct_late_by_origin_dest, #14
delayed_by_day #15
) = dask.compute(*aggregations)
print(datetime.now() - start)

### Visualization of Average Delay

We first take a look at the average delay for flights from the data set. Below, we look at the different representations for the average delay time for year, month, day, hour, and finally by carrier.

For the average delay by year, there dont seem to be many deviations. There are two significant years higher than 10 minutes in 2000 and 2007. We aloso see several years with low average delays less than 6 minutes including 1991, 1992, 2002, and 2003.

When we look at the average delay by month, we see December having the largest delay times with an average delay time of nearly 12 minutes. The month of September is the lowest month of delays. These highs and lows are most likely attributed to the flight patterns of people. Many people do not travel in September because school is starting back up and people have already taken time off for the summer. There are also a lot more travel conducted during December because of Christmas, one of the more popular holidays for travel.

An interesting plot to observe is the average delay by hour. It seems that there is a constant increase in delay times as the day gets going. From 6am to around 8pm, there is a constant increase in the average delay time. It starts less than 2 minutes on average for delays and increases all the way up to nearly 14 minutes. After 8pm, the time decreases again into the early morning, leveling back to "low" levels less than 4 minutes around 2am.

The average delay by day of the week does not have much to note. Delays are fairly stable around 8 minutes on average for each day, with day 5, Friday, being higher at 10 minutes. This is most likely due to high travel frequency on that day.

Finally, we see the average delay be carrier. A few of the first carriers in the plot have average delay times larger than 12 minutes. This decreases, but none of the carriers seem to fall below an average delay time of 8 minutes. This is most likely due to the fact that nearly every carrier has to deal with similar factors such as weather, airport issues, and leveraging the control tower for take off. No single carrier should have vastly different times than the others if all things are equal for every carrier.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.charts.attributes import cat
from bokeh.charts import Bar
from bokeh.layouts import gridplot

# Average Delay by Year
p1 = Bar(delayed_by_year.reset_index(), 'Year', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", 
         title="Average Delay by Year")

# Average Delay by Month
delayed_by_month = delayed_by_month.sort_index()
p2 = Bar(delayed_by_month.reset_index(), 'Month', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", 
         title="Average Delay by Month")

# Average Delay by Hour of Day
p3 = Bar(delayed_by_hour.reset_index(), 'Hour', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes",
         title="Average Delay by Hour of Day")

# Average Delay by Hour of Day
p4 = Bar(delayed_by_day.reset_index(), 'DayOfWeek', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes",
         title="Average Delay by Day of Week")

# Average Delay by Carrier
delayed_by_carrier = delayed_by_carrier.reset_index()
delayed_by_carrier['UniqueCarrier'] = delayed_by_carrier['UniqueCarrier'].astype('O')
p5 = Bar(delayed_by_carrier, label=cat('UniqueCarrier', sort=False), values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", xlabel="Unique Carrier", title="Average Delay by Carrier")


show(gridplot([[p1,p2],[p3,p4], [p5,None]], plot_width=400, plot_height=300))

## Which airports are most likely to be delayed flying out of or into?

The analysis conducted below looks at the combination of departure delays and arrival delays for each airport in the dataset. The average of these two values is what is used to determine which airport is the most likely to have delays.

From our analysis, we see that there are 7 airports that have an average delay in over 30% of their flights. This includes the largest delay we see at Moore County Airport (SOP) which averages delays 39.1% of the time. This is due to departure delays that occur 36.8% of the time and arrival delays that occur 41.4% of the time.

In [ ]:
airport_delays_pcts = (pd.merge(pct_delayed_by_origin, pct_late_by_dest, left_on='Origin', right_on='Dest')
                         .assign(AvgDelay= lambda x: (x['PercDepDelay'] + x['PercArrDelay'])/2)
                         .sort_values(by='AvgDelay', ascending=False)
                         .drop('Dest', axis=1)
                )

airport_delays_pcts = pd.merge(airport_delays_pcts, delayed_by_origin_count, on='Origin')

airport_delays_pcts[airport_delays_pcts['DepCount'] > 50].nlargest(15, 'AvgDelay')

## Which flights with same origin and destination are most likely to be delayed?

The next question we want to answer are which typical flights are more likely to be delayed. Using a similar process as conducted previously, we look at the departure delays and arrival delays and average the value to get the total value of delayed flights. For this question, however, we want all flights where the origin and destination are the same instead of looking at individual airports.

The worst flight seems to be the flights that fly from Jackson International (JAN) to Baton Rouge Metropolitan (BTR). This flight is delayed 64.6% of the time on average. The delays occur 58.2% of the time for departures and a significant 70.9% of the time for arrivals.

These result tells us that if you are taking this flight, or any flights that are high on this list, then you can most likely expect a delay, either leaving from the origin or arriving to the destination.

In [ ]:
org_dest_pcts = (pd.merge(pct_delayed_by_origin_dest, pct_late_by_origin_dest, on=['Origin','Dest'])
                 .assign(AvgDelay= lambda x: (x['PercDepDelay'] + x['PercArrDelay'])/2)
                 .sort_values(by='AvgDelay', ascending=False)
                )

org_dest_pcts = pd.merge(org_dest_pcts, delayed_by_origin_dest_count, on=['Origin','Dest'])

org_dest_pcts[org_dest_pcts['DepCount'] > 50].nlargest(15, 'AvgDelay')

In [ ]:
from bokeh.charts import Histogram

hist = Histogram(df[df['DepDelay']>15][['DepDelay']].sample(.25).compute().dropna(), 
                 values='DepDelay', bins=50)

show(hist)

The visualization above shows the counts of different delay times for a sample of 25% the flights. Many of the flights fall within the first 2 bins of delays less than 100 minutes, with 3.5 million being less than 50 minutes and aroudn 750,000 being between 50 and 100 minutes. The rest fall in the range of 100 to around 500. There may be further values higher than this, but there are so few flights delayed longer than 500 minutes (over 8 hours) that they do not show.

## Can you regress how delayed a flight will be before it is delayed?

## What are the most important features for this regression?

# Regression of Delay

The Dask module is a solution for processing "big data," however, the it currently does not include built in methods for regression or classification, like other big data solutions. The following will use a series of simple random sampling to a size that fits into a pandas dataframe to find the coefficient estimates of a linear model. The coefficients will be averaged to make a final prediction. This process also assists in not over fitting the model.

#### The following features will be explore to predict if the flight will have departure delay

##### The predicted variable will be: 
* Departure Delay (DepDelay)

##### The explanatory variables:
* Scheduled departure hour (Hour)
* Flight distance (Distance)
* Age of plane (Age)

In [ ]:
# Sample the entire data set as large as possible a few times. Each time has it's own cross validation sampling.
def sample_coef(Xcols, ycol, df, samp_size = .1, seeds = [123,456,789,101,112]):
    t = datetime.now()
    from sklearn import linear_model
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    import dask
    reg = linear_model.LinearRegression(n_jobs=-1)
    coefs = []
    
    for i in range(len(seeds)):
        start = datetime.now()
        # Take a sample from all the data
        all_cols = [ycol] + Xcols
        Xy = df[all_cols].sample(samp_size, random_state=seeds[i]).compute().dropna(axis=0)
        X = Xy[Xcols]
        y = Xy[ycol].values

        reg.fit(X, y)
        #print('Coefficients: \n', reg.coef_)
        coefs.append(reg.coef_)
        print("Time for Sample {}: ".format((i+1)), datetime.now() - start)
        #print(datetime.now() - start)
    
    del Xy, X, y

    coef_df = pd.DataFrame.from_records(coefs, columns=Xcols)
    coef_avg = coef_df.mean()
    print("\nCoefficients:")
    print(coef_df)    
    print("\nAverage Coefficients:")
    print(coef_avg)
    
    beta_cols = []
    for m, c in zip(coef_avg.index, coef_avg.values):
        b_col = "Beta_"+m
        df["Beta_"+m] = df[m]*c
        beta_cols.append(b_col)

    df['Predicted'] = df[beta_cols].sum(axis=1)

    #df['SqError'] = (df['Predicted'] - df[ycol])**2
    #mse = df[['SqError']].mean().compute()
    
    df_tmp = df[['Predicted']+[ycol]].sample(.4).compute().dropna()
    y_true = df_tmp[ycol]
    y_pred = df_tmp['Predicted']
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print("\nEstimated Evaluation Scores on 40%:")
    print("Mean Squared Error: ", mse)
    print("R Squared: ", r2)
    print("\nTotal Time: ", datetime.now() - t)
    return coef_df, coef_avg

In [ ]:
Xcols = ['Hour', 'Distance']
ycol =  'DepDelay'
coef_df1, coef_avg1 = sample_coef(Xcols, ycol, df)

In [ ]:
Xcols = ['Hour_scaled', 'Distance_scaled']
ycol =  'DepDelay'
coef_df2, coef_avg2 = sample_coef(Xcols, ycol, df)

In [ ]:
Xcols = ['Hour', 'Distance', 'Age']
ycol =  'DepDelay'
coef_df3, coef_avg3 = sample_coef(Xcols, ycol, df_with_tails)

In [ ]:
Xcols = ['Hour_scaled', 'Distance_scaled', 'Age_scaled']
ycol =  'DepDelay'
coef_df4, coef_avg4 = sample_coef(Xcols, ycol, df_with_tails)

In [ ]:
print("Total Run Time:", total_time - datetime.now())

### Regression Results

We conducted regression on the data set and wanted to determine what variables may affect the departure delay time the most. We conducted the analysis 5 times with different seeds to cross-validate and select different samples. Due to the size of the data set and the limitations of dask, we selected variables based on previous observations and visualizations. The main variables we wanted to observe were the hour the flight was supposed to take off and the distance it was to travel. We conducted this regression twice. The first run was prior to scaling the data to one another while the second took this scaling into account.

In our first run, we saw that hour was the dominant factor of these two values in predicting the time of delay for a flight. The hour variable had an average coefficient of 0.727 compared to the 0.001 value of distance. This is a skewed value however because the hour value ranges from 0 to 23 and the distance value can be quite large depending on the flight being conducted. To observe this relationship appropriately, we scale the hour and distance values. After doing this scaling, we see coeffiecients of 3.453 for the departure hour and 0.820 for the distance. This means that the delay time for departure can be calculated with a factor of 3.453 multiplied by the scaled hour and a factor of 0.82 multiplied by the scaled flight distance. The Mean Squared Error was 866.89, slightly larger than the non-scaled analysis of 804.55, with an R Squared value of -0.067.

Our second analysis added the age of the plane to the analysis. The age of the plane was represented in number of months and was calculated using the difference between the first recorded flight and the flight being conducted. In this analysis, we continued using the hour of departure and distance of the scheudled flight. When we sclaed the three variables, we saw coefficients of 3.938 for the hour of departure for the flight, 0.623 for the distance of the flight, and 0.154 for the age of the plane. This means that the later in the day the flight is, the longer it is going, and the older the plane is, the longer departure delay the flight will have. The hour plays the largest role in affecting the delay time when we use these coefficients. The Mean Squared Error of this regression for 40% of the data was 866.01, which is just slightly better than the initial regression using just hour of departure and distance of flight.

### Conclusion

Dask is a new "big data" alternative for those preferring the Python language. Although it is in active development by Continuum.io it still lacks certain features, such as, a drop-in generalized linear model. The dask dataframe is easy to use as it follow the popular pandas convention, however, unfortunately it does not allow for row slicing. Row slicing is convient for mini-batch operations that could have been usefull in this case. It is possible to do row slicing with dask arrays, but the data must be on disk using a compatible storage. Parquet is not compatible. Therefore, at this stage of the dask project, dask dataframes work well with out-of-core classification machine learning, and dask arrays are suitable for out-of-core regression for techniques that can utilize mini-batch operations. The dask-glm project looks to tackle out-of-core and distributed regression problems.

### Future Work

* Optimize with index key base on Data, deptarture time, and TailNum
* Use of alternative compression, such as snappy or LZ4
    * http://java-performance.info/performance-general-compression/
* Use a different big data approach to find a more efficient way to estimating the linear model coefficients:
    * Spark MLLib
    * Turi/Graphlab Create

## Bibliography

* Dask Documentation, http://dask.pydata.org/en/latest/
* Distributed Optimization and Statistical Learning via the Alternating Direction Method of Multipliers, Boyd, et al http://stanford.edu/~boyd/papers/pdf/admm_distr_stats.pdf
* https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp
* Variable Descriptions: http://stat-computing.org/dataexpo/2009/the-data.html
* Dask example using airline data https://jcrist.github.io/dask-sklearn-part-3.html

## Appendices

### Appendix A - CSV to Parquet Conversion

### Appendix B - Benchmark Tests

### Appendix C - Comparison of Dask Files
* Ryan's Hardware: 
    - CPU: Intel i5-4300M @ 2.60GHz
    - Disk: Samsung SSD 850 Pro
    - RAM: 8 GB
    

* Dask using original csv:
    - no conversion
    - size on disk
        - 11.2 gb
    - benchmark of describing 'Distance':
        - Approx. 4 minutes
* Dask using uncompressed parquet: 
    - conversion to parquet
        - approx 10 minutes
    - size on disk:
        - 13.8 gb
    - benchmark of describing 'Distance':
        - 1 loop, best of 3: 6.2 s per loop
* Dask using gzip compressed parquet:
    - converstion to parquet
        - approx 42 minutes
    - size on disk:
        - 1.36 gb <- big difference
    - benchmark of describing 'Distance':
        - 1 loop, best of 3: 8.83 s per loop

#### Summary
Dask allows for out of core management of data sets. CSV files are universal, but slow to process. Converting to parquet file format, speeds up the process by a factor of 38. Using the gzip compression, reduces size on disk from 13.8gb to 1.36 or about 10% of the uncompressed size. This comes in handy for a distributed processing in a cluster since not as much network bandwidth would be needed. The trade off of compression is a 42.4% increasing in processing time, however, 3 additional seconds is hardly noticable, but might be more of an issue for other tasks. 

### Appendix D - Processing, Creation, and Scaling of Features

In [ ]:
# Data Location
#parq_folder = "../data/parquet-tiny/" # Testing
#parq_folder = "../data/parquet_25/" # Higher Load Testing
parq_folder = "../data/parquet/" # Full data

df = dd.read_parquet(parq_folder)

# Create an hour field
# 2400 minutes from midnight reduced to 2399 then int division drops to 23
df = df.assign(Hour=df.CRSDepTime.astype(float).clip(upper=2399)//100) 

# Months from 0 AD
df['FlightAge'] = 12*df['Year']+df['Month']-1

# The months from the first recorded flight is consider the approx age of the plane. 
# Unfortunately, tail numbers not tracked until 1995. 

# Find the first year and month of a tail numbers flight history
tail_births = (df.groupby('TailNum')[['FlightAge']].min().reset_index()
                 .rename(columns={'FlightAge':'FirstFlight'}))

df_with_tails = dd.merge(df[df['Year']>1994], tail_births, how='left', on='TailNum')
df_with_tails['Age'] = df_with_tails['FlightAge'] - df_with_tails['FirstFlight']

#df_with_tails = df_with_tails.drop(['FlightAge','FirstFlight'], axis=1)

start = datetime.now()
def scaler(df, column):
    return (df[column] - df[column].dropna().mean())/df[column].dropna().std()

# Scale columns for regression of all data
df['Hour_scaled'] = scaler(df, 'Hour')
df['Distance_scaled'] = scaler(df, 'Distance')

# Scale columns for regression for after 1994
df_with_tails['Hour_scaled'] = scaler(df_with_tails, 'Hour')
df_with_tails['Distance_scaled'] = scaler(df_with_tails, 'Distance')
df_with_tails['Age_scaled'] = scaler(df_with_tails, 'Age')

print("Time to Build: ", datetime.now() - start)

start = datetime.now()
df.to_parquet(parq_folder+"processed/", compression='gzip', object_encoding='utf8')
print("Time to make processed data: ", datetime.now() - start)
df_with_tails.to_parquet(parq_folder+"processed_age/", compression='gzip', object_encoding='utf8')
print("Time to make processed data with plane ages: ", datetime.now() - start)

Time to Build:  0:00:00.093601
